In [1]:
%pip install numpy pandas matplotlib python-pptx

Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys  # noqa
# sys.path.append('/workspace/forecasting_simulation')  # noqa

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import seaborn as sns 
import matplotlib.colors as mcolors
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from utils import * 

## LOAD DATA

In [2]:
import pandas as pd

# Load the data
data = pd.read_excel('Mcbride-LP.xlsx')
data['datetime'] = pd.to_datetime(data['datetime'])  # Ensure datetime is in correct format

# Check the number of columns in the DataFrame
num_columns = data.shape[1]

# Column indices: 8th column (index 7) for kwh and 9th column (index 8) for kw
kwh_column_idx = 7  # 8th column (0-based index)
kw_column_idx = 8   # 9th column (0-based index)

# Check datetime interval to determine if it's 5-minute or 1-hour data
time_diff = data['datetime'].diff().dt.total_seconds().median()
frequency = 12 if time_diff == 300 else 1  # 5-minute interval if median diff is 300 seconds, otherwise 1 hour

# Handling 5-minute data
if frequency == 12:  # 5-minute interval
    print("Data is 5-minute interval.")

    # Check if there are 8 or 9 columns
    if num_columns == 8:
        # Check if the 8th column is named as 'kw'
        if data.columns[kwh_column_idx].lower() == 'kw':
            # Rename it to 'kwh' and create 'kw' as the 9th column
            data.rename(columns={data.columns[kwh_column_idx]: 'kwh'}, inplace=True)
            data['kw'] = data['kwh'] * 12  # Create the 9th column by multiplying kwh by 12
        else:
            # If it's already kwh, just create 'kw' column
            data['kw'] = data.iloc[:, kwh_column_idx] * 12  # Create the 9th column by multiplying kwh by 12
            data.rename(columns={data.columns[kwh_column_idx]: 'kwh'}, inplace=True)
            
        # Ensure the data has 9 columns now
        print(f"Data now has {data.shape[1]} columns (9 columns expected).")

    elif num_columns == 9:
        # Simply rename the 8th and 9th columns as kwh and kw
        data.rename(columns={data.columns[kwh_column_idx]: 'kwh', data.columns[kw_column_idx]: 'kw'}, inplace=True)

# Handling 1-hour data
elif frequency == 1:  # 1-hour interval
    print("Data is 1-hour interval.")

    # Check if there are 8 or 9 columns
    if num_columns == 8:
        # Copy the value from the 8th column (kwh) to the 9th column (kw)
        data['kw'] = data.iloc[:, kwh_column_idx]
        # Rename the columns appropriately
        data.rename(columns={data.columns[kwh_column_idx]: 'kwh', 'kw': 'kw'}, inplace=True)
        print(f"Data now has {data.shape[1]} columns (9 columns expected).")

    elif num_columns == 9:
        # Ensure the 8th and 9th columns are the same
        if not (data.iloc[:, kwh_column_idx] == data.iloc[:, kw_column_idx]).all():
            print("Warning: 8th and 9th columns do not match.")
        # Rename the columns as kwh and kw
        data.rename(columns={data.columns[kwh_column_idx]: 'kwh', data.columns[kw_column_idx]: 'kw'}, inplace=True)

# Show first few rows to verify the results
print(data.head())


Data is 5-minute interval.
Data now has 9 columns (9 columns expected).
  supply period            datetime       date  correct hour  hour  weekday  \
0        Nov-23 2023-10-26 00:00:00 2023-10-26             0     1        3   
1        Nov-23 2023-10-26 00:05:00 2023-10-26             0     1        3   
2        Nov-23 2023-10-26 00:10:00 2023-10-26             0     1        3   
3        Nov-23 2023-10-26 00:15:00 2023-10-26             0     1        3   
4        Nov-23 2023-10-26 00:20:00 2023-10-26             0     1        3   

       wesm    kwh      kw  
0  3.337600  39.08  468.96  
1  4.897380  39.08  468.96  
2  4.825615  39.08  468.96  
3  4.000234  39.00  468.00  
4  4.003252  39.00  468.00  


In [3]:
# Check the columns in your data
print(data.columns)

Index(['supply period', 'datetime', 'date', 'correct hour', 'hour', 'weekday',
       'wesm', 'kwh', 'kw'],
      dtype='object')


In [4]:
import pandas as pd

def generate_energy_summary(data, datetime_column='datetime', kwh_column='kwh', kw_column='kw'):
    # Ensure 'kwh' and 'kw' columns are numeric
    data[kwh_column] = pd.to_numeric(data[kwh_column], errors='coerce')
    data[kw_column] = pd.to_numeric(data[kw_column], errors='coerce')

    # Determine the frequency based on the time intervals in the datetime column
    interval_seconds = data[datetime_column].diff().dt.total_seconds().median()
    frequency = 12 if interval_seconds == 300 else 1  # 12 for 5 minutes, 1 for 1 hour

    # Group by supply period and aggregate the necessary columns
    energy_summary = data.groupby("supply period", sort=False).agg({
        "supply period": "count",
        kwh_column: "sum",
        kw_column: "max"
    })

    # Rename columns for readability
    energy_summary.columns = ["number of intervals", "kwh", "kw"]

    # Calculate 'number of hours' based on frequency and drop 'number of intervals'
    energy_summary["number of hours"] = energy_summary["number of intervals"] / frequency
    energy_summary.drop(columns=["number of intervals"], inplace=True)

    # Add a total row with the sum of kWh and the max of kW
    energy_summary.loc["Total"] = energy_summary.sum(numeric_only=True)
    energy_summary.loc["Total", "kw"] = energy_summary.iloc[:-1]["kw"].max()

    # Calculate the load factor
    energy_summary["load factor"] = (energy_summary["kwh"] / (energy_summary['kw'] * energy_summary["number of hours"])) * 100

    # Create a copy of the summary data without the Total row for statistics
    original_energy_summary = energy_summary.iloc[:-1].copy()

    # Add Average, Max, Min rows based on the original data (excluding 'Total')
    energy_summary.loc["Average"] = original_energy_summary.mean(numeric_only=True)
    energy_summary.loc["Max"] = original_energy_summary.max(numeric_only=True)
    energy_summary.loc["Min"] = original_energy_summary.min(numeric_only=True)

    # Reset index to remove 'supply period' from being the index
    energy_summary = energy_summary.reset_index()

    # Reorder columns as specified
    energy_summary = energy_summary[["supply period", "number of hours", "kwh", "kw", "load factor"]]

    return energy_summary

# Generate energy summary
energy_summary = generate_energy_summary(data, kwh_column='kwh', kw_column='kw')

In [5]:
print(energy_summary.dtypes)

supply period       object
number of hours    float64
kwh                float64
kw                 float64
load factor        float64
dtype: object


In [6]:
import psycopg2
import pandas as pd

def connect_to_database(host="localhost", database="solx", user="postgres", password="123456", port="5432"):
    try:
        # Establishing the connection
        conn = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            port=port
        )
        print("Database connection successful")
        return conn
    except psycopg2.OperationalError as e:
        print(f"Error connecting to the database: {e}")
        return None

def fetch_data_for_du(du_name, return_dataframe=False, host="localhost", database="solx", user="postgres", password="123456", port="5432"):
    # Connect to the PostgreSQL database
    conn = connect_to_database(host, database, user, password, port)
    if not conn:
        return None

    cursor = conn.cursor()

    # Step 1: Find the du_id for the specified du_name
    cursor.execute("""
        SELECT du_id 
        FROM du_rates.distribution_utilities 
        WHERE du_name = %s
    """, (du_name,))
    
    # Check if the query returned a valid du_id
    du_id_result = cursor.fetchone()
    if du_id_result is None:
        print(f"No du_id found for du_name: {du_name}")
        cursor.close()
        conn.close()
        return None

    du_id = du_id_result[0]
    print(f"Fetched du_id: {du_id} for {du_name}")

    # Step 2: Retrieve data from effective_rates for the specified du_id
    cursor.execute("""
        SELECT du_id, supply_period, effective_rate
        FROM du_rates.effective_rates 
        WHERE du_id = %s
        ORDER BY supply_period
    """, (du_id,))
    
    results = cursor.fetchall()
    if not results:
        print(f"No data found in effective_rates for du_name: {du_name}")
        cursor.close()
        conn.close()
        return None

    if return_dataframe:
        # Create a DataFrame from the results
        data = []
        for row in results:
            formatted_date = row[1].strftime('%b-%y')  # Format as 'Jan-22'
            data.append([row[0], formatted_date, row[2]])

        # Convert list of lists into DataFrame
        df = pd.DataFrame(data, columns=["du_id", "supply_period", "effective_rate"])

        cursor.close()
        conn.close()

        return df
    else:
        # Print the results if not returning a DataFrame
        print(f"Data for du_name '{du_name}':")
        for row in results:
            print(row)

        cursor.close()
        conn.close()

# Example usage for printing data:
fetch_data_for_du("MERALCO", return_dataframe=False)

Database connection successful
Fetched du_id: 59 for MERALCO
Data for du_name 'MERALCO':
(59, datetime.date(2022, 1, 1), 5.4262)
(59, datetime.date(2022, 2, 1), 5.1957)
(59, datetime.date(2022, 3, 1), 5.4737)
(59, datetime.date(2022, 4, 1), 5.8724)
(59, datetime.date(2022, 5, 1), 6.2277)
(59, datetime.date(2022, 6, 1), 6.559)
(59, datetime.date(2022, 7, 1), 6.7756)
(59, datetime.date(2022, 8, 1), 6.5812)
(59, datetime.date(2022, 9, 1), 6.9393)
(59, datetime.date(2022, 10, 1), 6.9192)
(59, datetime.date(2022, 11, 1), 6.9917)
(59, datetime.date(2022, 12, 1), 6.7975)
(59, datetime.date(2023, 1, 1), 7.1291)
(59, datetime.date(2023, 2, 1), 6.9154)
(59, datetime.date(2023, 3, 1), 7.379)
(59, datetime.date(2023, 4, 1), 7.3295)
(59, datetime.date(2023, 5, 1), 7.6697)
(59, datetime.date(2023, 6, 1), 7.2502)
(59, datetime.date(2023, 7, 1), 6.6066)
(59, datetime.date(2023, 8, 1), 6.3929)
(59, datetime.date(2023, 9, 1), 6.8252)
(59, datetime.date(2023, 10, 1), 7.1267)
(59, datetime.date(2023, 11, 

In [7]:
# Example usage for getting data as a DataFrame:
df = fetch_data_for_du("MERALCO", return_dataframe=True)
if df is not None:
    print(df)
else:
    print("No data returned.")

Database connection successful
Fetched du_id: 59 for MERALCO
    du_id supply_period  effective_rate
0      59        Jan-22          5.4262
1      59        Feb-22          5.1957
2      59        Mar-22          5.4737
3      59        Apr-22          5.8724
4      59        May-22          6.2277
5      59        Jun-22          6.5590
6      59        Jul-22          6.7756
7      59        Aug-22          6.5812
8      59        Sep-22          6.9393
9      59        Oct-22          6.9192
10     59        Nov-22          6.9917
11     59        Dec-22          6.7975
12     59        Jan-23          7.1291
13     59        Feb-23          6.9154
14     59        Mar-23          7.3790
15     59        Apr-23          7.3295
16     59        May-23          7.6697
17     59        Jun-23          7.2502
18     59        Jul-23          6.6066
19     59        Aug-23          6.3929
20     59        Sep-23          6.8252
21     59        Oct-23          7.1267
22     59        No

In [8]:
def combine_data(energy_summary, df):
    # Rename columns in energy_summary for consistency
    energy_summary.rename(columns={'supply period': 'supply_period'}, inplace=True)
    
    # Clean energy_summary by removing rows with 'Total', 'Average', 'Max', or 'Min' in 'supply_period'
    energy_summary = energy_summary[~energy_summary['supply_period'].isin(['Total', 'Average', 'Max', 'Min'])]
    
    # Convert 'supply_period' to datetime to handle chronological ordering correctly
    energy_summary['supply_period'] = pd.to_datetime(energy_summary['supply_period'], format='%b-%y')

    # Ensure df also has 'supply_period' in datetime format
    df['supply_period'] = pd.to_datetime(df['supply_period'], format='%b-%y')

    # Get the minimum and maximum supply period from energy_summary
    min_supply_period = energy_summary['supply_period'].min().strftime('%b-%y')
    max_supply_period = energy_summary['supply_period'].max().strftime('%b-%y')

    print(f"Min Supply Period: {min_supply_period}, Max Supply Period: {max_supply_period}")

    # Filter df to only include rows within the range of energy_summary supply periods
    df_filtered = df[(df['supply_period'] >= energy_summary['supply_period'].min()) & (df['supply_period'] <= energy_summary['supply_period'].max())]

    # Merge the energy_summary and filtered df on supply_period
    merged_df = pd.merge(energy_summary, df_filtered, on='supply_period', how='left')

    # Check columns in merged_df to ensure 'kwh' and 'effective_rate' are present
    print("Columns in merged_df:", merged_df.columns)

    # Fill missing effective_rate in merged_df with the previous available effective_rate
    merged_df['effective_rate'] = merged_df['effective_rate'].fillna(method='ffill')

    # Return the result with 'supply_period', 'kwh', and 'effective_rate'
    result_df = merged_df[['supply_period', 'kwh', 'effective_rate']]
    
    return result_df

# Example usage:
# Assuming you have 'df' from your database and 'energy_summary' DataFrame
result_df = combine_data(energy_summary, df)

# Print the final result
print(result_df)

Min Supply Period: Nov-23, Max Supply Period: Sep-24
Columns in merged_df: Index(['supply_period', 'number of hours', 'kwh', 'kw', 'load factor', 'du_id',
       'effective_rate'],
      dtype='object')
   supply_period         kwh  effective_rate
0     2023-11-01  371827.200          7.1938
1     2023-12-01  370132.620          6.5332
2     2024-01-01  373162.260          6.6468
3     2024-02-01  404063.915          7.1020
4     2024-03-01  380537.250          6.7502
5     2024-04-01  366739.350          6.3889
6     2024-05-01  366741.180          6.8344
7     2024-06-01  396093.750          5.0036
8     2024-07-01  359701.770          7.0057
9     2024-08-01  343199.400          6.9554
10    2024-09-01  355670.310          6.9554


C:\Users\MARK\AppData\Local\Temp\ipykernel_6240\1385687637.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  energy_summary['supply_period'] = pd.to_datetime(energy_summary['supply_period'], format='%b-%y')
C:\Users\MARK\AppData\Local\Temp\ipykernel_6240\1385687637.py:30: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df['effective_rate'] = merged_df['effective_rate'].fillna(method='ffill')


In [9]:
print("Columns in df:", result_df.columns)

Columns in df: Index(['supply_period', 'kwh', 'effective_rate'], dtype='object')


In [10]:
print(result_df.dtypes)

supply_period     datetime64[ns]
kwh                      float64
effective_rate           float64
dtype: object


In [11]:
# Print the entire result_df
print(result_df[['supply_period', 'kwh', 'effective_rate']])

   supply_period         kwh  effective_rate
0     2023-11-01  371827.200          7.1938
1     2023-12-01  370132.620          6.5332
2     2024-01-01  373162.260          6.6468
3     2024-02-01  404063.915          7.1020
4     2024-03-01  380537.250          6.7502
5     2024-04-01  366739.350          6.3889
6     2024-05-01  366741.180          6.8344
7     2024-06-01  396093.750          5.0036
8     2024-07-01  359701.770          7.0057
9     2024-08-01  343199.400          6.9554
10    2024-09-01  355670.310          6.9554


In [12]:
print(result_df['kwh'].isna().sum())  # This will show how many NaN values are in the kwh column
print(result_df['kwh'])  # This will display the raw data to help identify issues

0
0     371827.200
1     370132.620
2     373162.260
3     404063.915
4     380537.250
5     366739.350
6     366741.180
7     396093.750
8     359701.770
9     343199.400
10    355670.310
Name: kwh, dtype: float64


In [13]:
def calculate_percentage_discount(result_df, floor_price, du_discount):
    # Ensure 'effective_rate' is a numeric column
    result_df['effective_rate'] = pd.to_numeric(result_df['effective_rate'], errors='coerce')
    
    # Convert kwh to numeric, filling NaNs with 0 if needed
    result_df['kwh'] = pd.to_numeric(result_df['kwh'], errors='coerce').fillna(0)

    # Calculate the discounted price using percentage
    result_df['DU Discounted Price'] = result_df['effective_rate'] - (result_df['effective_rate'] * (du_discount / 100))
    result_df['DU Discounted Price'] = result_df['DU Discounted Price'].clip(lower=floor_price).round(4)

    # Format the discounted price as a string with 4 decimal places (for display purposes)
    result_df['DU Discounted Price'] = result_df['DU Discounted Price'].apply(lambda x: f"{x:.4f}")

    # Convert 'DU Discounted Price' back to numeric for calculations
    result_df['DU Discounted Price'] = pd.to_numeric(result_df['DU Discounted Price'], errors='coerce')

    # Calculate Total Charges
    result_df['Total Charges'] = result_df['kwh'] * result_df['DU Discounted Price']
    
    # Calculate the average DU rate
    du_rate_ave = result_df['effective_rate'].mean()
    result_df['DU Rate Ave'] = f"{du_rate_ave:.4f}"

    # Calculate the average discounted price using percentage
    discounted_price_ave = du_rate_ave - (du_rate_ave * (du_discount / 100))
    discounted_price_ave = max(discounted_price_ave, floor_price)
    result_df['DU Discounted Price Ave'] = f"{discounted_price_ave:.4f}"

    # Format 'kwh' and 'Total Charges' with commas and decimal places
    result_df['kwh'] = result_df['kwh'].apply(lambda x: f"{x:,.3f}")
    result_df['Total Charges'] = result_df['Total Charges'].apply(lambda x: f"{x:,.2f}")

    print("Fixed Discount DataFrame:\n", result_df)  # Debugging print

    return result_df

In [14]:
# Example usage with result_df
floor_price = 6.5  # Set your floor price
du_discount = 3   # Set your DU discount (percentage or fixed)

# Choose either fixed or percentage discount calculation:
# If you want to apply fixed discount:
result_df_fixed = calculate_percentage_discount(result_df, floor_price, du_discount)

Fixed Discount DataFrame:
    supply_period          kwh  effective_rate  DU Discounted Price  \
0     2023-11-01  371,827.200          7.1938               6.9780   
1     2023-12-01  370,132.620          6.5332               6.5000   
2     2024-01-01  373,162.260          6.6468               6.5000   
3     2024-02-01  404,063.915          7.1020               6.8889   
4     2024-03-01  380,537.250          6.7502               6.5477   
5     2024-04-01  366,739.350          6.3889               6.5000   
6     2024-05-01  366,741.180          6.8344               6.6294   
7     2024-06-01  396,093.750          5.0036               6.5000   
8     2024-07-01  359,701.770          7.0057               6.7955   
9     2024-08-01  343,199.400          6.9554               6.7467   
10    2024-09-01  355,670.310          6.9554               6.7467   

   Total Charges DU Rate Ave DU Discounted Price Ave  
0   2,594,610.20      6.6699                  6.5000  
1   2,405,862.03      

In [15]:
import pandas as pd

# Assuming result_df contains 'supply_period', 'kwh', and 'effective_rate'

def calculate_fixed_discount(result_df, floor_price, du_discount): 
    # Ensure 'effective_rate' is a numeric column
    result_df['effective_rate'] = pd.to_numeric(result_df['effective_rate'], errors='coerce')
    
    # Convert kwh to numeric, filling NaNs with 0 if needed
    result_df['kwh'] = pd.to_numeric(result_df['kwh'], errors='coerce').fillna(0)
    
    # Calculate the discounted price
    result_df['DU Discounted Price'] = result_df['effective_rate'] - du_discount
    result_df['DU Discounted Price'] = result_df['DU Discounted Price'].clip(lower=floor_price).round(4)
    
    # Format the discounted price as a string with 4 decimal places (for display purposes)
    result_df['DU Discounted Price'] = result_df['DU Discounted Price'].apply(lambda x: f"{x:.4f}")
    
    # Convert 'DU Discounted Price' back to numeric for calculations
    result_df['DU Discounted Price'] = pd.to_numeric(result_df['DU Discounted Price'], errors='coerce')
    
    # Calculate Total Charges
    result_df['Total Charges'] = result_df['kwh'] * result_df['DU Discounted Price']
    
    # Calculate the average DU rate
    du_rate_ave = result_df['effective_rate'].mean()
    result_df['DU Rate Ave'] = f"{du_rate_ave:.4f}"

    # Calculate the average discounted price
    discounted_price_ave = du_rate_ave - du_discount
    discounted_price_ave = max(discounted_price_ave, floor_price)
    result_df['DU Discounted Price Ave'] = f"{discounted_price_ave:.4f}"
    
    # Format 'kwh' and 'Total Charges' with commas and decimal places
    result_df['kwh'] = result_df['kwh'].apply(lambda x: f"{x:,.3f}")
    result_df['Total Charges'] = result_df['Total Charges'].apply(lambda x: f"{x:,.2f}")

    print("Fixed Discount DataFrame:\n", result_df)  # Debugging print

    return result_df

In [16]:
# Example usage with result_df
floor_price = 6.5  # Set your floor price
du_discount = 3   # Set your DU discount (percentage or fixed)

# Choose either fixed or percentage discount calculation:
# If you want to apply fixed discount:
result_df_fixed = calculate_fixed_discount(result_df, floor_price, du_discount)

Fixed Discount DataFrame:
    supply_period    kwh  effective_rate  DU Discounted Price Total Charges  \
0     2023-11-01  0.000          7.1938                  6.5          0.00   
1     2023-12-01  0.000          6.5332                  6.5          0.00   
2     2024-01-01  0.000          6.6468                  6.5          0.00   
3     2024-02-01  0.000          7.1020                  6.5          0.00   
4     2024-03-01  0.000          6.7502                  6.5          0.00   
5     2024-04-01  0.000          6.3889                  6.5          0.00   
6     2024-05-01  0.000          6.8344                  6.5          0.00   
7     2024-06-01  0.000          5.0036                  6.5          0.00   
8     2024-07-01  0.000          7.0057                  6.5          0.00   
9     2024-08-01  0.000          6.9554                  6.5          0.00   
10    2024-09-01  0.000          6.9554                  6.5          0.00   

   DU Rate Ave DU Discounted Price A